## Выявление релевантных пар исследователей и формирование команд для совместной научной работы на основе семантического анализа текстов научных статей.

Постановка задачи:

Предполагаю сравнивать схожесть научных интересов путём анализа статей, которые пишет научный сотрудник. По результатам анализа можно рекомендовать совместную работу двух исследователей, рекомендовать научных руководителей аспирантам и собирать научные группы, специализирующиеся на определённом направлении.

### Теоретические задачи:
1. Провести анализ того что уже сделано в этом направлении. Что можно рассмотреть:
 - методы кластеризации текстов
 - извлечение информации (например, поиск удк по шаблону)
 - рекомендательные системы на схожести интересов
2. Посмотреть научные статьи по теме
3. Нарисовать модель работы будущей системы

### Практические задачи:
1. Собрать данные:
 - сделать веб-скрапинг для киберленинки
 - ? собрать названия дипломных работ
 - ? собрать связку преподаватель - какими работами руководил
2. Обработка данных
 - лемматизация, очистка от стоп слов 
 - проверка на выбросы, визуализация
 - посмотреть статистики, схожести текстов и любых других параметров которые на этом этапе я откопаю (TF-IDF)
 - извлечь полезную информации об авторе, учреждении, основных мыслей статьи (можно построить облако слов, облако близких тем, как в последней недоделанной домашке) 
 - векторизовать названия статей, текстов, и всего остального текста 
3. Классическое машинное обучение
 - задача регрессии: вводим название конференции, и высвечивается вероятность с которой данный человек будет в ней участвовать. Можно тут линейные модели обкатать, логистическую регрессию например.
 - задача классификации: группируем людей по научным областям на основе названий их работ, здесь два варианта, широкая кластеризация (по фундаментальным наукам) и узкая (внутри одной науки). Обучить ставить удк?
 - кластеризация, пытаемся выделить людей со схожими научными интересами без изначально заданных областей знаний, что потом ляжет в основу рекомендаций для научных групп
 - меряем качество этого всего измеряем (все метрики забыла уже мда)
 - ? автоматическая оценка эффективности работы научного сотрудника, ранжирование (для начала по количеству написанных работ, но можно ещё оценивать "серьёзность" проведённого исследования, над мерилом "серьёзности" надо подумать, хотя звучит как антиплагиат номер два)
4. Рекомендательные системы
 - вводим id (или ФИО) преподавателя, и высвечиваются рекомендуемые ему конференции и научные журналы, тип "user" 
 - вводим название конференеции, и высвечиваются юзеры, которым это было бы интересно, тип "item"
 - ? вводим тему диплома человека, высвечиваются по убыванию потенциальные дипломные руководители

## Парсинг

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [6]:
from bs4 import BeautifulSoup
import requests

def parse_journal_volume(url):
    # Загружаем HTML со страницы
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки HTTP
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке страницы: {e}")
        return None

    # Парсим HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Извлекаем данные
    volume_data = {
        'volume_title': soup.find('h1', class_='journal-volume-title').get_text(strip=True),
        'articles': []
    }

    for article in soup.find_all('div', class_='journal-article'):
        # Извлекаем заголовок и ссылку
        title_block = article.find('div', class_='journal-article-title')
        title = title_block.get_text(strip=True) if title_block else None
        link = title_block.find('a')['href'] if title_block and title_block.find('a') else None

        # Извлекаем авторов
        authors = article.find('div', class_='journal-article-authors').get_text(strip=True) \
            if article.find('div', class_='journal-article-authors') else None

        # Обрабатываем аннотацию
        annotation = article.find('div', class_='journal-article-annotation').get_text(strip=True) \
            .replace('Аннотация: ', '') if article.find('div', class_='journal-article-annotation') else None

        # Обрабатываем ключевые слова
        keywords_block = article.find('div', class_='journal-article-keywords')
        keywords = keywords_block.get_text(strip=True).replace('Ключевые слова: ', '').split(', ') \
            if keywords_block else []

        volume_data['articles'].append({
            'title': title,
            'link': link,
            'authors': authors,
            'annotation': annotation,
            'keywords': keywords
        })

    return volume_data

# Пример использования
if __name__ == "__main__":
    journal_url = "https://itstd-journal.ru/?page_id=636&volume=2&year=2025"
    result = parse_journal_volume(journal_url)
    
    if result:
        print(f"Название выпуска: {result['volume_title']}")
        print(f"Найдено статей: {len(result['articles'])}\n")
        
        for idx, article in enumerate(result['articles'], 1):
            print(f"Статья #{idx}:")
            print(f"Заголовок: {article['title']}")
            print(f"Ссылка: {article['link']}")
            print(f"Авторы: {article['authors']}")
            print(f"Ключевые слова: {', '.join(article['keywords'])}\n")
    else:
        print("Не удалось получить данные")

Название выпуска: Выпуск № 2 (2025)
Найдено статей: 4

Статья #1:
Заголовок: АКТУАЛЬНЫЕ ЗАДАЧИ МАТЕМАТИЧЕСКОГО ОБЕСПЕЧЕНИЯ  СТАНДАРТИЗАЦИИ ИНФОРМАЦИОННЫХ ТЕХНОЛОГИЙ
Ссылка: /?page_id=1080&article=334
Авторы: Головин С.А., Лоцманов А.Н., Тихомиров С.Г.
Ключевые слова: Ключевые слова:ИТ стандартизация, умный (SMART) стандарт, эталонная модель, информационные 
технологии, системный подход, умный (SMART) документ

Статья #2:
Заголовок: ОСОБЕННОСТИ И ПЕРСПЕКТИВЫ ПРИМЕНЕНИЯ ANFIS СТРУКТУР ДЛЯ ПОСТРОЕНИЯ СИСТЕМ УПРАВЛЕНИЯ  МОДУЛЬНЫМИ МАНИПУЛЯЦИОННЫМИ РОБОТАМИ
Ссылка: /?page_id=1080&article=333
Авторы: Быковцев Ю.А., Лохин В.М., Манько С.В., Смирнов А.В.
Ключевые слова: Ключевые слова:модульный манипуляционный робот, динамическая коррекция, интеллектуальное управление, 
нейро-нечеткие системы, ANFIS

Статья #3:
Заголовок: ПОДХОДЫ К ОДНОКЛАССОВОЙ КЛАССИФИКАЦИИ  РЕГУЛЯРНЫХ ВЫРАЖЕНИЙ
Ссылка: /?page_id=1080&article=332
Авторы: Демидов Н.А.
Ключевые слова: Ключевые слова:регулярное выражение, патте

In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm  # для прогресс-бара

def parse_single_volume(url):
    """Парсит одну страницу журнала и возвращает список статей"""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Ошибка при загрузке {url}: {str(e)}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []

    for article in soup.find_all('div', class_='journal-article'):
        # Извлекаем данные с проверкой наличия элементов
        title_block = article.find('div', class_='journal-article-title')
        title = title_block.get_text(strip=True) if title_block else None
        link = title_block.find('a')['href'] if title_block and title_block.find('a') else None
        
        authors_block = article.find('div', class_='journal-article-authors')
        authors = authors_block.get_text(strip=True) if authors_block else None
        
        annotation_block = article.find('div', class_='journal-article-annotation')
        annotation = annotation_block.get_text(strip=True).replace('Аннотация: ', '') \
            if annotation_block else None

        articles.append({
            'title': title,
            'annotation': annotation,
            'authors': authors,
            'link': link
        })
    
    return articles

def create_journal_df(urls):
    """Создает DataFrame из списка URL журналов"""
    all_articles = []
    
    # Используем tqdm для отображения прогресса
    for url in tqdm(urls, desc="Парсинг журналов"):
        articles = parse_single_volume(url)
        all_articles.extend(articles)
    
    return pd.DataFrame(all_articles)

# Список URL для парсинга
journal_urls = [
    'https://itstd-journal.ru/?page_id=636&volume=4&year=2017',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2017',
'https://itstd-journal.ru/?page_id=636&volume=2&year=2017', 
'https://itstd-journal.ru/?page_id=636&volume=1&year=2017',
'https://itstd-journal.ru/?page_id=636&volume=4&year=2016',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2016',
'https://itstd-journal.ru/?page_id=636&volume=2&year=2016',
'https://itstd-journal.ru/?page_id=636&volume=1&year=2016',
'https://itstd-journal.ru/?page_id=636&volume=4&year=2015',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2015',
'https://itstd-journal.ru/?page_id=636&volume=2&year=2015',
'https://itstd-journal.ru/?page_id=636&volume=1&year=2015',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2014',
'https://itstd-journal.ru/?page_id=636&volume=1&year=2025',
'https://itstd-journal.ru/?page_id=636&volume=4&year=2024',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2024',
'https://itstd-journal.ru/?page_id=636&volume=2&year=2024',
'https://itstd-journal.ru/?page_id=636&volume=1&year=2024',
'https://itstd-journal.ru/?page_id=636&volume=4&year=2023',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2023',
'https://itstd-journal.ru/?page_id=636&volume=2&year=2023',
'https://itstd-journal.ru/?page_id=636&volume=1&year=2023',
'https://itstd-journal.ru/?page_id=636&volume=4&year=2022',
'https://itstd-journal.ru/?page_id=636&volume=3&year=2022',
'https://itstd-journal.ru/?page_id=636&volume=2&year=2022',
'https://itstd-journal.ru/?page_id=636&volume=1&year=2022',
'https://itstd-journal.ru/?page_id=636&volume=4&year=2021'
]

# Создаем DataFrame
df = create_journal_df(journal_urls)

# Просмотр результата
print(f"\nНайдено статей: {len(df)}")
print("\nПример данных:")
df.head(3)


Парсинг журналов: 100%|████████████████████████████████████████████████████████████████| 27/27 [00:26<00:00,  1.03it/s]


Найдено статей: 203

Пример данных:


,title,annotation,authors,link
0,НА ПОВЕСТКЕ ДНЯ – ПОСТРОЕНИЕ НООСФЕРЫ,Аннотация:Показа ны потенциал новых информацио...,Назаренко А.П.; Сарьян В.К.; Мещеряков В.Р.,/?page_id=1080&article=111
1,ОПЫТ ИНТЕГРАЦИИ РАСПРЕДЕЛЕННЫХ ИНФОРМАЦИОННЫХ ...,Аннотация:Рассматривается опыт разработок архи...,Денисов В.Ф.; Куделькин В.А.,/?page_id=1080&article=107
2,ПУТИ РЕШЕНИЯ НЕКОТОРЫХ ПРОБЛЕМ КОМПЛЕКСНОЙ БЕЗ...,Аннотация:Сформулированы приоритетные проблемы...,Костогрызов А.И.,/?page_id=1080&article=110


In [22]:
#сохранение
df.to_csv('it-standart.csv', index=False, encoding='utf-8-sig')

In [24]:
df = pd.read_csv('it-standart.csv', delimiter=',') #тестирую открытие файла
df

,title,annotation,authors,link
0,НА ПОВЕСТКЕ ДНЯ – ПОСТРОЕНИЕ НООСФЕРЫ,Аннотация:Показа ны потенциал новых информацио...,Назаренко А.П.; Сарьян В.К.; Мещеряков В.Р.,/?page_id=1080&article=111
1,ОПЫТ ИНТЕГРАЦИИ РАСПРЕДЕЛЕННЫХ ИНФОРМАЦИОННЫХ ...,Аннотация:Рассматривается опыт разработок архи...,Денисов В.Ф.; Куделькин В.А.,/?page_id=1080&article=107
2,ПУТИ РЕШЕНИЯ НЕКОТОРЫХ ПРОБЛЕМ КОМПЛЕКСНОЙ БЕЗ...,Аннотация:Сформулированы приоритетные проблемы...,Костогрызов А.И.,/?page_id=1080&article=110
3,ПРОГНОЗИРОВАНИЕ ВРЕМЕНИ ДО НАРУШЕНИЯ ЦЕЛОСТНОС...,Аннотация:Предложен подход к прогнозированию о...,Нистратов А.А.; Нистратов Г.А.,/?page_id=1080&article=105
4,ОЦЕНКА СООТВЕТСТВИЯ В ОБЛАСТИ ТЕХНОЛОГИЙ ИСКУС...,Аннотация:Технологии искусственного интеллекта...,Гарбук С.В.,/?page_id=1080&article=109
...,...,...,...,...
198,ОБЛАСТИ ПРИМЕНИМОСТИ НЕСЕКВЕНЦИАЛЬНЫХ АЛГОРИТМОВ,Аннотация:Цель работы – исследование области п...,"Козлов А.В., Титов Е.К.",/?page_id=1080&article=241
199,ПРИОРИТЕТНЫЕ НАПРАВЛЕНИЯ РАЗВИТИЯ СИСТЕМНОЙ ИН...,Аннотация:Настоящая работа посвящена формирова...,"Зацаринный А.А., Костогрызов А.И., Нистратов А.А.",/?page_id=1080&article=243
200,ТЕОРЕТИЧЕСКОЕ РАЗВИТИЕ МОДЕЛЕЙ ДЛЯ ОЦЕНКИ ЗАЩИ...,Аннотация:Рассмотрены вопросы использования ан...,"Гусев К. В., Леонтьев А.С.",/?page_id=1080&article=242
201,АНАЛИЗ ДАННЫХ О ПОЕЗДКАХ В МЕГАПОЛИСЕ И ПРОГНО...,Аннотация:В данной статье исследуется набор да...,"Дзержинский Р.И., Лапшин И.А., Аносов Т.Э.",/?page_id=1080&article=240


In [26]:
!pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/e6/c4/d2e09fbc937d1f76baae34e526662cc718e23a904321bf4a40282d190033/pdfplumber-0.11.6-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/42.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/42.8 kB ? eta -:--:--
     -------------------------------------- 42.8/42.8 kB 345.9 kB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six==20250327 from https://files.pythonhosted.org/packages/29/2f/409e174b5a0195aa6a814c7359a1285f1c887a4c84aff17ed03f607c06ba/pdfminer_six-20250327-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/a4/f3/8d3a350efb4286b5ebdabcf6736f51d8e3b10dbe68804c6930b00f5cf329/pypdfium2-4.30.1-py3-none-w

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import BytesIO
import re
import pdfplumber
from tqdm import tqdm
import time

def extract_pdf_text(pdf_url):
    """Скачивает и извлекает текст из PDF с обработкой ошибок"""
    try:
        response = requests.get(pdf_url, timeout=15)
        response.raise_for_status()
        
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            full_text = '\n'.join(page.extract_text() for page in pdf.pages if page.extract_text())

        # Находим границы текста
        start_match = re.search(r'Keywords:.*?\n', full_text, re.IGNORECASE | re.DOTALL)
        end_match = re.search(r'Список литературы', full_text, re.IGNORECASE)

        start_idx = start_match.end() if start_match else 0
        end_idx = end_match.start() if end_match else len(full_text)
        
        return full_text[start_idx:end_idx].strip()

    except Exception as e:
        print(f"Ошибка при обработке {pdf_url}: {str(e)}")
        return None

def enrich_with_article_text(df):
    """Добавляет текст статей в DataFrame"""
    df['article_text'] = None
    pdf_links = []

    # Собираем ссылки на PDF
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Поиск PDF-ссылок"):
        try:
            page_response = requests.get(row['link'], timeout=10)
            soup = BeautifulSoup(page_response.text, 'html.parser')
            pdf_link = soup.find('a', {'class': 'download-button'})['href']
            pdf_links.append(pdf_link)
            time.sleep(1)  # Задержка между запросами
        except:
            pdf_links.append(None)

    # Парсим текст PDF
    df['pdf_link'] = pdf_links
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Обработка PDF"):
        if row['pdf_link']:
            df.at[idx, 'article_text'] = extract_pdf_text(row['pdf_link'])
    
    return df.drop(columns=['pdf_link'])

# Загрузка существующего DataFrame (пример)
#df = pd.read_csv('journal_articles.csv')

# Пример использования с тестовыми данными
test_df = pd.DataFrame([{
    'title': 'Тестовая статья',
    'link': 'https://itstd-journal.ru/?page_id=1080&article=334',
    'authors': 'Иванов И.И.',
    'annotation': 'Пример аннотации'
}])

# Обогащаем данные текстами статей
enriched_df = enrich_with_article_text(test_df)

# Результат
print("\nОбразец данных с текстом статьи:")
print(enriched_df[['title', 'article_text']].head())


Обработка PDF:   0%|                                                                             | 0/1 [00:00<?, ?it/s]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF: 100%|█████████████████████████████████████████████


Образец данных с текстом статьи:
             title                                       article_text
0  Тестовая статья  document.\nВведение\nВ настоящее время техноло...


In [37]:
enriched_df

,title,link,authors,annotation,article_text
0,Тестовая статья,https://itstd-journal.ru/?page_id=1080&article...,Иванов И.И.,Пример аннотации,document.\nВведение\nВ настоящее время техноло...


In [38]:
enriched_df.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [39]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import BytesIO
import re
import pdfplumber
from tqdm import tqdm
import time
from urllib.parse import urljoin

# Базовый URL сайта
BASE_URL = "https://itstd-journal.ru"

def extract_pdf_text(pdf_url):
    """Скачивает и извлекает текст из PDF с обработкой ошибок"""
    try:
        response = requests.get(pdf_url, timeout=15)
        response.raise_for_status()
        
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            full_text = '\n'.join(page.extract_text() for page in pdf.pages if page.extract_text())

        # Находим границы текста
        start_match = re.search(r'Keywords:.*?\n', full_text, re.IGNORECASE | re.DOTALL)
        end_match = re.search(r'Список литературы', full_text, re.IGNORECASE)

        start_idx = start_match.end() if start_match else 0
        end_idx = end_match.start() if end_match else len(full_text)
        
        return full_text[start_idx:end_idx].strip()

    except Exception as e:
        print(f"Ошибка при обработке {pdf_url}: {str(e)}")
        return None

def enrich_with_article_text(df):
    """Добавляет текст статей в DataFrame"""
    df['article_text'] = None
    pdf_links = []

    # Собираем ссылки на PDF
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Поиск PDF-ссылок"):
        try:
            # Формируем полный URL статьи
            article_url = urljoin(BASE_URL, row['link'])
            
            page_response = requests.get(article_url, timeout=10)
            soup = BeautifulSoup(page_response.text, 'html.parser')
            
            # Ищем PDF ссылку
            pdf_relative = soup.find('a', {'class': 'download-button'})['href']
            pdf_absolute = urljoin(BASE_URL, pdf_relative)
            
            pdf_links.append(pdf_absolute)
            time.sleep(1)  # Задержка между запросами
        except Exception as e:
            print(f"Ошибка для статьи {row['title']}: {str(e)}")
            pdf_links.append(None)

    # Парсим текст PDF
    df['pdf_link'] = pdf_links
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Обработка PDF"):
        if row['pdf_link']:
            df.at[idx, 'article_text'] = extract_pdf_text(row['pdf_link'])
    
    return df.drop(columns=['pdf_link'])

# Загрузка данных из CSV
df = pd.read_csv('it-standart.csv')

# Фикс относительных ссылок (двойная проверка)
df['link'] = df['link'].apply(lambda x: urljoin(BASE_URL, x) if not x.startswith('http') else x)

# Обогащаем данные текстами статей
enriched_df = enrich_with_article_text(df)

print(f"Обработка завершена. Сохранено {len(enriched_df)} статей.")

Обработка PDF:   0%|                                                                           | 0/203 [00:00<?, ?it/s]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:   0%|▎                                                                  | 1/203 [00:03<10:06,  3.00s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaB

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:   6%|███▉                                                              | 12/203 [00:33<08:57,  2.81s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:   6%|████▏                                                             | 13/203 [00:36<08:32,  2.70s/

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  11%|███████▍                                                          | 23/203 [01:08<11:07,  3.71s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  12%|███████▊                                                          | 24/203 [01:10<09:21,  3.14s/it]CropBox missing from /Page, defaulting to MediaB

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  18%|███████████▋                                                      | 36/203 [01:41<07:59,  2.87s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missin

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  24%|███████████████▌                                                  | 48/203 [02:23<06:24,  2.48s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  24%|███████████████▉                                                  | 49/203 [02:26<06:41,  2.61s/it]CropBox missing from /Page, defaulting to MediaB

...

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  87%|████████████████████████████████████████████████████████▋        | 177/203 [09:36<01:01,  2.38s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missin

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  93%|████████████████████████████████████████████████████████████▏    | 188/203 [10:07<00:45,  3.03s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missin

CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  96%|██████████████████████████████████████████████████████████████▍  | 195/203 [10:32<00:30,  3.76s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF:  97%|██████████████████████████████████████████████████████████████▊  | 196/203 [10:34<00:22,  3.23s/it]CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaB

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Обработка PDF: 100%|█████████████████████████████████████████████████████████████████| 203/203 [11:01<00:00,  3.26s/it]

Обработка завершена. Сохранено 203 статей.


In [70]:
enriched_df.to_json('it-st-articles.json', orient='records')

In [2]:
df = pd.read_json('it-st-articles.json')
df

,title,annotation,authors,link,article_text
0,НА ПОВЕСТКЕ ДНЯ – ПОСТРОЕНИЕ НООСФЕРЫ,Аннотация:Показа ны потенциал новых информацио...,Назаренко А.П.; Сарьян В.К.; Мещеряков В.Р.,https://itstd-journal.ru/?page_id=1080&article...,НА ПОВЕСТКЕ ДНЯ – ПОСТРОЕНИЕ НООСФЕРЫ\n1Назаре...
1,ОПЫТ ИНТЕГРАЦИИ РАСПРЕДЕЛЕННЫХ ИНФОРМАЦИОННЫХ ...,Аннотация:Рассматривается опыт разработок архи...,Денисов В.Ф.; Куделькин В.А.,https://itstd-journal.ru/?page_id=1080&article...,ОПЫТ ИНТЕГРАЦИИ\nРАСПРЕДЕЛЕННЫХ ИНФОРМАЦИОННЫХ...
2,ПУТИ РЕШЕНИЯ НЕКОТОРЫХ ПРОБЛЕМ КОМПЛЕКСНОЙ БЕЗ...,Аннотация:Сформулированы приоритетные проблемы...,Костогрызов А.И.,https://itstd-journal.ru/?page_id=1080&article...,ПУТИ РЕШЕНИЯ НЕКОТОРЫХ ПРОБЛЕМ КОМПЛЕКСНОЙ\nБЕ...
3,ПРОГНОЗИРОВАНИЕ ВРЕМЕНИ ДО НАРУШЕНИЯ ЦЕЛОСТНОС...,Аннотация:Предложен подход к прогнозированию о...,Нистратов А.А.; Нистратов Г.А.,https://itstd-journal.ru/?page_id=1080&article...,ПРОГНОЗИРОВАНИЕ ВРЕМЕНИ ДО НАРУШЕНИЯ ЦЕЛОСТНОС...
4,ОЦЕНКА СООТВЕТСТВИЯ В ОБЛАСТИ ТЕХНОЛОГИЙ ИСКУС...,Аннотация:Технологии искусственного интеллекта...,Гарбук С.В.,https://itstd-journal.ru/?page_id=1080&article...,ОЦЕНКА СООТВЕТСТВИЯ В ОБЛАСТИ ТЕХНОЛОГИЙ\nИСКУ...
...,...,...,...,...,...
198,ОБЛАСТИ ПРИМЕНИМОСТИ НЕСЕКВЕНЦИАЛЬНЫХ АЛГОРИТМОВ,Аннотация:Цель работы – исследование области п...,"Козлов А.В., Титов Е.К.",https://itstd-journal.ru/?page_id=1080&article...,"subsidiary computations, information construct..."
199,ПРИОРИТЕТНЫЕ НАПРАВЛЕНИЯ РАЗВИТИЯ СИСТЕМНОЙ ИН...,Аннотация:Настоящая работа посвящена формирова...,"Зацаринный А.А., Костогрызов А.И., Нистратов А.А.",https://itstd-journal.ru/?page_id=1080&article...,1 Введение\nСравнительный анализ различных рос...
200,ТЕОРЕТИЧЕСКОЕ РАЗВИТИЕ МОДЕЛЕЙ ДЛЯ ОЦЕНКИ ЗАЩИ...,Аннотация:Рассмотрены вопросы использования ан...,"Гусев К. В., Леонтьев А.С.",https://itstd-journal.ru/?page_id=1080&article...,"function, approximation, unauthorized access.\..."
201,АНАЛИЗ ДАННЫХ О ПОЕЗДКАХ В МЕГАПОЛИСЕ И ПРОГНО...,Аннотация:В данной статье исследуется набор да...,"Дзержинский Р.И., Лапшин И.А., Аносов Т.Э.",https://itstd-journal.ru/?page_id=1080&article...,УДК 519.24\nАНАЛИЗ ДАННЫХ О ПОЕЗДКАХ В МЕГАПОЛ...


In [7]:
from tqdm import tqdm
import re

#добавлю ещё год статьи
def extract_publication_year(full_url):
    """Извлекает год публикации из полного URL статьи"""
    try:
        response = requests.get(full_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Поиск в заголовке выпуска
        volume_title = soup.find('div', class_='journal-volume-title')
        if volume_title:
            match = re.search(r'\((\d{4})\)', volume_title.get_text(strip=True))
            if match:
                return int(match.group(1))
        
        # Резервный поиск в цитировании
        citation = soup.find('div', class_='journal-citation')
        if citation:
            match = re.search(r'– (\d{4})\. –', citation.get_text())
            if match:
                return int(match.group(1))
        
        return None
    
    except Exception as e:
        print(f"Ошибка при обработке {full_url}: {str(e)}")
        return None

def add_publication_years(df):
    """Добавляет годы публикации в DataFrame"""
    df['year'] = None
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Извлечение годов"):
        try:
            # Используем полную ссылку напрямую
            df.at[idx, 'year'] = extract_publication_year(row['link'])
            time.sleep(0.5)  # Защита от блокировки
        except Exception as e:
            print(f"Ошибка для статьи '{row['title']}': {str(e)}")
    
    return df

# Загрузка данных (пример)
df = pd.read_json('it-st-articles.json')


# Добавляем годы публикации
updated_df = add_publication_years(df)

# Результат
print("\nОбразец данных с годом:")
print(updated_df[['title', 'year']].head())

Извлечение годов: 100%|██████████████████████████████████████████████████████████████| 203/203 [04:13<00:00,  1.25s/it]


Образец данных с годом:
                                               title  year
0              НА ПОВЕСТКЕ ДНЯ – ПОСТРОЕНИЕ НООСФЕРЫ  2017
1  ОПЫТ ИНТЕГРАЦИИ РАСПРЕДЕЛЕННЫХ ИНФОРМАЦИОННЫХ ...  2017
2  ПУТИ РЕШЕНИЯ НЕКОТОРЫХ ПРОБЛЕМ КОМПЛЕКСНОЙ БЕЗ...  2017
3  ПРОГНОЗИРОВАНИЕ ВРЕМЕНИ ДО НАРУШЕНИЯ ЦЕЛОСТНОС...  2017
4  ОЦЕНКА СООТВЕТСТВИЯ В ОБЛАСТИ ТЕХНОЛОГИЙ ИСКУС...  2017


In [9]:
updated_df.to_json('it-st-articles-with-year.json', orient='records') 